# Using ALBERT for Question Answering - SQuAD

In [1]:
#export
import os

In [2]:
os.chdir("../")

In [3]:
#export
from transformers import AutoTokenizer,PretrainedConfig
import numpy as np
import pandas as pd
import pickle
import re
import requests
import json
from src import *
import fire

# Loading the data

In [4]:
#export
def load_dfs(config):
    """
    - loads train and valid csv files, coverts them into dataframes
    - truncates the dataset if testing
    - drops those which exceed maximum allowed sequence length 
    """
    train = pd.read_csv(config.data_path+f"/train_{config.squad_version}_{config.model_name}.csv")
    valid = pd.read_csv(config.data_path+f"/val_{config.squad_version}_{config.model_name}.csv")

    train.drop_duplicates(inplace=True)
    valid.drop_duplicates(inplace=True)

    # randomizing the order of training data
    train = train.sample(frac=1).reset_index(drop=True) #random_state = config.seed
    valid = valid.sample(frac=1).reset_index(drop=True)

    # reduce df sizes if testing
    if config.testing:
        train = train[:int(len(train)/config.data_reduction)]
        valid = valid[:int(len(valid)/config.data_reduction)]

    return remove_max_sl(train, config.max_seq_len), remove_max_sl(valid, config.max_seq_len)

# Creating Dataloaders

In [5]:
#export
def make_dataloaders(config, train_df, valid_df):
    """
    - preprocesses raw text input into numericalized tensors 
    - creates train and valid dataloaders out of dataframes
    """
    tok = AutoTokenizer.from_pretrained(config.model)
    
    proc_tok = QATokenizerProcessor(tok.tokenize, config.max_seq_len, config.start_tok, config.end_tok)

    vocab = {tok.convert_ids_to_tokens(i):i for i in range(tok.vocab_size)}
    proc_num = QANumericalizeProcessor(vocab, unk_tok_idx=config.unk_idx)
    proc_qa = QALabelProcessor(str2tensor,config.adjustment)

    if (not (os.path.exists(config.data_path+f"/squad_{config.squad_version}_data_trn.pkl"))) or config.recreate_ds or config.testing:
        il_train = SquadTextList.from_df(train_df,config.feat_cols,config.label_cols,config.sep_tok)
        il_valid = SquadTextList.from_df(valid_df,config.feat_cols,config.label_cols,config.sep_tok)

        ll_valid = LabeledData(il_valid,il_valid.labels,proc_x = [proc_tok,proc_num], proc_y=[proc_qa])
        ll_train = LabeledData(il_train,il_train.labels,proc_x = [proc_tok,proc_num], proc_y=[proc_qa])

        # saving/loading presaved data if not testing
        if not config.testing:
            # save an object
            pickle.dump(ll_train, open( config.data_path+f"/squad_{config.squad_version}_data_trn.pkl", "wb" ) )
            pickle.dump(ll_valid, open( config.data_path+f"/squad_{config.squad_version}_data_val.pkl", "wb" ) )
    else:
        # load an object
        ll_train = pickle.load( open( config.data_path+f"/squad_{config.squad_version}_data_trn.pkl", "rb" ) )
        ll_valid = pickle.load( open( config.data_path+f"/squad_{config.squad_version}_data_val.pkl", "rb" ) )

    collate_fn = partial(pad_collate_qa,pad_idx=config.pad_idx)
    
    train_sampler = SortishSampler(ll_train.x, key=lambda t: len(ll_train[int(t)][0]), bs=config.bs)
    train_dl = DataLoader(ll_train, batch_size=config.bs, sampler=train_sampler, collate_fn=collate_fn)
    
    valid_sampler = SortSampler(ll_valid.x, key=lambda t: len(ll_valid[int(t)][0]))
    valid_dl = DataLoader(ll_valid, batch_size=config.bs, sampler=valid_sampler, collate_fn=collate_fn)

    return DataBunch(train_dl,valid_dl)

# Defining Model & Learner

In [6]:
#export
def get_learner(config, data, opt_func):
    """
    - defines the model and relevant callbacks
    - creates learner object used to train the model
    """
    model_kwargs = {"pretrained_model_name_or_path": config.weights}
    
    if not config.load_checkpoint: model_kwargs["askai_config"] = config
    model = AlbertForQuestionAnsweringMTL.from_pretrained(**model_kwargs)

    # setting up callbacks
    cbfs = [partial(QAAvgStatsCallback,[acc_qa,acc_pos,exact_match,f1_score]),
            ProgressCallback,
            Recorder]
    
    if torch.cuda.is_available(): cbfs.append(CudaCallbackMTL)

    if not config.testing and config.save_checkpoint:
        cbfs.append(partial(SaveModelCallback,save_model_qa,config.output_dir,config.model,config.squad_version))

    if config.effective_bs and config.bs != config.effective_bs:
        cbfs.append(partial(GradientAccumulation,config.bs,config.effective_bs))

    if config.stats_update_freq is not None: cbfs.append(partial(TrainStatsCallback,config.stats_update_freq))

    learn = Learner(model, data, cross_entropy_qa_mtl_wtd,lr=config.max_lr,cb_funcs=cbfs,splitter=albert_splitter,\
                opt_func=opt_func)
    return learn

In [7]:
config = Config(
    data_path = "../data/SQuAD/2.0", # replace with the directory containing the parsed csv files
    output_dir = "./models", # for storing model weights between epochs
    task = "SQuAD",
    squad_version = "2.0",
    testing=False,
    data_reduction = 1000, # reduce df sizes by this amount while testing 
    seed = 2020,
    model = "albert-base-v2",
    max_lr=3e-5,
    max_lr_last = 5e-5,
    phases = .3,
    optimizer="lamb", # choose between 'adam' or 'lamb'
    epochs=1,
    use_fp16=False,
    recreate_ds=False,
    bs=4, 
    effective_bs=4, # set this different from bs to determine gradient accumulation steps (i.e. effective_bs/bs)
    max_seq_len=512,
    start_tok = "[CLS]",
    end_tok = "[SEP]",
    sep_tok = "[SEP]",
    unk_idx=1,
    sep_idx=3,
    pad_idx=0,
    feat_cols = ["question","paragraph"],
    label_cols = ["idxs","is_impossible"],
    adjustment = 2,
    save_checkpoint = True,
    load_checkpoint=None,#"2020-02-07_14_58_00.025453-albert-base-v2-acc-0.61-ep-0-squad_2.0",
    num_labels_clas = 2,
    clas_dropout_prob = .1,
    stats_update_freq = .05,
)

In [8]:
#export
def main(config, return_learner=False):
    """
    - loads data
    - sets the Learning Rate and Momentum Scheduler 
    - defines the optimizer
    - trains and outputs results periodically 
    """
    if isinstance(config, str): config = Config(**json.load(open(config,"r")))
    assert type(config) == Config, f"config parameter type must be Config or a path to a json file"
    if config.effective_bs: 
        assert config.effective_bs >= config.bs, f"mini bs ({config.bs}) cannot be smaller than effective bs ({config.effective_bs})"
        assert config.effective_bs % config.bs == 0, "mini bs ({config.bs}) should be a factor of the effective bs ({config.effective_bs})"
    
    config.model_name=re.findall(r"(.+?)-",config.model)[0]
    config.weights=config.output_dir+f"/{config.load_checkpoint}" if config.load_checkpoint else config.model

    train,valid = load_dfs(config)
    data = make_dataloaders(config, train, valid)

    # set LR scheduler
    disc_lr_sched = sched_1cycle([config.max_lr,config.max_lr_last], config.phases)

    # set optimizer
    assert config.optimizer.lower() in ["adam","lamb"], f"invalid optimizer in config {config.optimizer}"
    opt_func = lamb_opt() if config.optimizer.lower() == "lamb" else adam_opt()

    learn = get_learner(config, data, opt_func)
    learn.fit(config.epochs)#,cbs=disc_lr_sched)
    if return_learner: return learn

In [10]:
learn = main(config, True) 

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-spiece.model from cache at /home/devkosal/.cache/torch/transformers/dd1588b85b6fdce1320e224d29ad062e97588e17326b9d05a0b29ee84b8f5f93.c81d4deb77aec08ce575b7a39a989a79dd54f321bfb82c2b54dd35f52f8182cf


dropping 216 out of 130294 questions which exceed max sequence length
dropping 124 out of 16315 questions which exceed max sequence length


INFO:transformers.configuration_utils:loading configuration file ./models/2020-02-07_14_58_00.025453-albert-base-v2-acc-0.61-ep-0-squad_2.0/config.json
INFO:transformers.configuration_utils:Model config {
  "adjustment": 2,
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bs": 4,
  "clas_dropout_prob": 0.1,
  "data_path": "../data/SQuAD/2.0",
  "data_reduction": 1000,
  "down_scale_factor": 1,
  "effective_bs": 4,
  "embedding_size": 128,
  "end_tok": "[SEP]",
  "epochs": 1,
  "feat_cols": [
    "question",
    "paragraph"
  ],
  "finetuning_task": null,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "label_cols": [
    "idxs",
    "is_impossible"
  ],
  "layer_norm_eps": 1e-12,


epoch,train_loss,train_acc_qa,train_acc_pos,train_exact_match,train_f1_score,valid_loss,valid_acc_qa,valid_acc_pos,valid_exact_match,valid_f1_score,time


INFO:root:epoch 0 stats for iter 1626 out of 32520 iters are : {'loss': 2.9555501902675276, 'acc_qa': tensor(0.7448, device='cuda:0'), 'acc_pos': tensor(0.8630, device='cuda:0'), 'exact_match': tensor(0.6221, device='cuda:0'), 'f1_score': 0.8193979983273446}
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

INFO:root:epoch 0 stats for iter 13008 out of 32520 iters are : {'loss': 3.008898670529674, 'acc_qa': tensor(0.7427, device='cuda:0'), 'acc_pos': tensor(0.8692, device='cuda:0'), 'exact_match': tensor(0.6234, device='cuda:0'), 'f1_score': 0.8211594657496761}
INFO:root:epoch 0 stats for iter 14634 out of 32520 iters are : {'loss': 2.9967891020056032, 'acc_qa': tensor(0.7429, device='cuda:0'), 'acc_pos': tensor(0.8

In [11]:
#export
if __name__ == "__main__":
    fire.Fire(main)

ERROR: The function received no value for the required argument: config
Usage: ipykernel_launcher.py CONFIG <flags>
  optional flags:        --return_learner

For detailed information on this command, run:
  ipykernel_launcher.py --help


FireExit: 2

/home/devkosal/anaconda3/envs/fastai/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
! python scripts/notebook2script.py  notebooks/"Train Albert on SQuAD.ipynb"